In [ ]:
import urllib
from urllib.request import urlopen
from urllib.request import Request, urlopen

from urllib.parse import urlencode
from urllib.request import urlopen
from datetime import datetime
import pandas as pd
import numpy as np
import csv
    
import os

from IPython.display import clear_output


import os

import pywt
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=FutureWarning)

# TCM, CTCM

In [ ]:
from numpy.linalg import solve


def my_method_1(index_list, len_wind, step_smooth):
    prognoz=[]
    
    train=index_list
#     iteratation_5=int((len(index_list)-len_wind)/step_smooth)+1
    iteratation_5=1

    itog_prog=[]
    for i in range(0,iteratation_5):
#         train=index_list[0+step_smooth*i:len_wind+step_smooth*i] 



        t=len(train)

        y1=[]
        y2=[]
        y3=[]
        y4=[]

        y_ist=[]
        y_ist.append(train[0])

        y_prog=[]
        y_prog.append(train[0])
        y_prog.append(train[1])


        y1.append(train[0])#
        y2.append(0)#Изменение
        y3.append(train[0]-y_prog[0])# ошибка прошлого прогноза
        y4.append(0)# ошибка изменения


        for i in range(1,t-1):

            y_ist.append(train[i+1])
            y1.append(train[i])
            y2.append(train[i]-train[i-1])#izmeneniya
            y3.append(train[i]-y_prog[i])#err
            y4.append(train[i]-train[i-1]-y_prog[i]+y_prog[i-1])#err izmeneniya

            y=np.array([y1,
                y2,
                y3,
                y4])

            m_y=np.dot(y,y.T)

            if i<3:
                m_y=m_y+np.diag(np.full(4,1))
            else:
                m_y=m_y  

        #     y_t=y1[-1]


            mb_y=np.dot(y,y_ist)

            resh=solve(m_y, mb_y)

            y_prog.append(resh[0]*y1[-1]+resh[1]*y2[-1]+resh[2]*y3[-1]+resh[3]*y4[-1])

        #     print(i)

        i=t-1
        y_ist.append(train[t-1])
        y1.append(train[i])
        y2.append(train[i]-train[i-1])#izmeneniya
        y3.append(train[i]-y_prog[i])#err
        y4.append(train[i]-train[i-1]-y_prog[i]+y_prog[i-1])#err izmeneniya

        y=np.array([y1,
                y2,
                y3,
                y4])

        m_y=np.dot(y,y.T)

        if i<3:
            m_y=m_y+np.diag(np.full(4,1))
        else:
            m_y=m_y  

        #     y_t=y1[-1]


        mb_y=np.dot(y,y_ist)

        resh=solve(m_y, mb_y)

        y_prog.append(resh[0]*y1[-1]+resh[1]*y2[-1]+resh[2]*y3[-1]+resh[3]*y4[-1])
        prognoz.append(y_prog[-1])
    return prognoz


In [ ]:
def my_method_1_correct(index_list, len_wind, step_smooth, persentile):
    prognoz=[]

    train=index_list
#     iteratation_5=int((len(index_list)-len_wind)/step_smooth)+1
    iteratation_5=1

    prognoz_correct=[]
    itog_prog=[]
    for i in range(0,iteratation_5):
#         train=index_list[0+step_smooth*i:len_wind+step_smooth*i] 



        t=len(train)

        y1=[]
        y2=[]
        y3=[]
        y4=[]

        y_ist=[]
        y_ist.append(train[0])

        y_prog=[]
        y_prog.append(train[0])
        y_prog.append(train[1])


        y1.append(train[0])#
        y2.append(0)#Изменение
        y3.append(train[0]-y_prog[0])#
        y4.append(0)# 

        tmp_prog=[]
        for i in range(1,t-1):

            y_ist.append(train[i+1])
            y1.append(train[i])
            y2.append(train[i]-train[i-1])#izmeneniya
            y3.append(train[i]-y_prog[i])#err
            y4.append(train[i]-train[i-1]-y_prog[i]+y_prog[i-1])#err izmeneniya

            y=np.array([y1,
                y2,
                y3,
                y4])

            m_y=np.dot(y,y.T)

            if i<3:
                m_y=m_y+np.diag(np.full(4,1))
            else:
                m_y=m_y  

        #     y_t=y1[-1]


            mb_y=np.dot(y,y_ist)

#             resh=solve(m_y, mb_y)
            while True:
                try:
                    resh=solve(m_y, mb_y)
                    break
                except Exception as exc:
                    resh=solve(m_y+np.diag(np.full(4,1)), mb_y)
                    break

            y_prog.append(resh[0]*y1[-1]+resh[1]*y2[-1]+resh[2]*y3[-1]+resh[3]*y4[-1])
            tmp_prog.append(resh[0]*y1[-1]+resh[1]*y2[-1]+resh[2]*y3[-1]+resh[3]*y4[-1])


        tmp_mape_vec=((np.array(tmp_prog)-np.array(train[-len(tmp_prog)-1:-1]))/np.array(train[-len(tmp_prog)-1:-1]))
        tmp_mape=tmp_mape_vec.mean()
        i=t-1
        y_ist.append(train[t-1])
        y1.append(train[i])
        y2.append(train[i]-train[i-1])#izmeneniya
        y3.append(train[i]-y_prog[i])#err
        y4.append(train[i]-train[i-1]-y_prog[i]+y_prog[i-1])#err izmeneniya

        y=np.array([y1,
                y2,
                y3,
                y4])

        m_y=np.dot(y,y.T)

        if i<3:
            m_y=m_y+np.diag(np.full(4,1))
        else:
            m_y=m_y  

        #     y_t=y1[-1]


        mb_y=np.dot(y,y_ist)

        while True:
            try:
                resh=solve(m_y, mb_y)
    #                 all_fr=pd.concat([all_fr,tmp])
    #                 print(i+'_'+j)
                break
            except Exception as exc:
                resh=solve(m_y+np.diag(np.full(4,1)), mb_y)
                break
#         resh=solve(m_y+np.diag(np.full(4,1)), mb_y) 
        
#         resh=solve(m_y, mb_y)

        y_prog.append(resh[0]*y1[-1]+resh[1]*y2[-1]+resh[2]*y3[-1]+resh[3]*y4[-1])


        prognoz.append(y_prog[-1])
        prognoz_correct.append(y_prog[-1]*(1+np.percentile(tmp_mape_vec, persentile)))#
#         prognoz_correct.append(y_prog[-1]*(1+np.percentile(np.array(pd.DataFrame(tmp_mape_vec).dropna()), persentile)))#
    return prognoz, prognoz_correct

# with wavelet

In [ ]:
import pywt

len_wind=399
step_smooth=1
# index_list=frame_3.NI.dropna()
wavelet='bior5.5'
modes='smooth'

def perebor_levels_wavedec_without_noise_naiv_1(index_list, len_wind, step_smooth, wavelet, modes):
# for kk in range(0,1):
    dates=index_list.index
#     prog1=[]
#     prog11=[]
    levl=pywt.dwt_max_level(len_wind, wavelet)
    if levl>7:
        levl=7
    
    iteratation_5=int((len(index_list)-len_wind)/step_smooth)
#     iteratation_5=5
    name=[]
    frame_prog_tcm=pd.DataFrame()
    frame_prog_ctcm=pd.DataFrame()
    for k in range(1,levl+1):
        
        lvl=k
        prog1=[]
        prog11=[]
#         toch=[]
        date=[]
        date1=[]
    
#         naiv=[]
        name.append(wavelet+'_'+modes+'_'+str(lvl))
        for i in range(0,iteratation_5):
    
            train=index_list[0+step_smooth*i:len_wind+step_smooth*i]
            A2= pywt.wavedec(train, wavelet, level=lvl, mode=modes)
            razl=A2[0]
            tmp=my_method_1_correct(razl, len_wind, step_smooth, persentile=0.8)
        
        
#         TCM
            arr=[ x for x in range(lvl+1)]
            arr[0]=np.append(razl,tmp[0][0])[1:]
            
            for j in range(1,lvl+1):
                arr[j]=None
            coeff_new=arr
            denoised_index = pywt.waverec(coeff_new, wavelet, mode=modes)
            date.append(dates[0+step_smooth*i:len_wind+1+step_smooth*i][-1:][0])
            prog1.append(denoised_index[-1])
            
#         CTCM
            arr1=[ x for x in range(lvl+1)]
            arr1[0]=np.append(razl,tmp[1][0])[1:]
            
            for j in range(1,lvl+1):
                arr1[j]=None
            coeff_new1=arr1
            denoised_index1 = pywt.waverec(coeff_new1, wavelet, mode=modes)
            date1.append(dates[0+step_smooth*i:len_wind+1+step_smooth*i][-1:][0])
            prog11.append(denoised_index1[-1])
            
            
        frame_prog_tcm[wavelet+'_'+modes+'_'+str(k)]=prog1
        frame_prog_ctcm[wavelet+'_'+modes+'_'+str(k)]=prog11
    frame_prog_tcm.index=date
    frame_prog_ctcm.index=date1
    return frame_prog_tcm, frame_prog_ctcm

def toch_f(index_list, len_wind, step_smooth):
    dates=index_list.index    
    iteratation_5=int((len(index_list)-len_wind)/step_smooth)
    frame_prog=pd.DataFrame()
    toch=[]
    date=[]
    
    for i in range(0,iteratation_5):
        train=index_list[0+step_smooth*i:len_wind+step_smooth*i]
        toch.append(index_list[0+step_smooth*i:len_wind+1+step_smooth*i][-1:].values[0])
        date.append(dates[0+step_smooth*i:len_wind+1+step_smooth*i][-1:][0])
    frame_prog['orig']=toch
    frame_prog.index=date
    return frame_prog    

In [ ]:
modes=['symmetric',
       'reflect',
       'smooth',
       'constant']

wavelets=['haar',
 'db1',
 'db2',
 'db3',
 'db4',
 'db5',
 'db6',
 'db7',
 'db8',
 'db9',
 'db10',
 'db11',
 'db12',
 'db13',
 'db14',
 'db15',
 'db16',
 'db17',
 'db18',
 'db19',
 'db20',
 'db21',
 'db22',
 'db23',
 'db24',
 'db25',
 'db26',
 'db27',
 'db28',
 'db29',
 'db30',
 'db31',
 'db32',
 'db33',
 'db34',
 'db35',
 'db36',
 'db37',
 'db38',
 'sym2',
 'sym3',
 'sym4',
 'sym5',
 'sym6',
 'sym7',
 'sym8',
 'sym9',
 'sym10',
 'sym11',
 'sym12',
 'sym13',
 'sym14',
 'sym15',
 'sym16',
 'sym17',
 'sym18',
 'sym19',
 'sym20',
 'coif1',
 'coif2',
 'coif3',
 'coif4',
 'coif5',
 'coif6',
 'coif7',
 'coif8',
 'coif9',
 'coif10',
 'coif11',
 'coif12',
 'coif13',
 'coif14',
 'coif15',
 'coif16',
 'coif17',
 'bior1.1',
 'bior1.3',
 'bior1.5',
 'bior2.2',
 'bior2.4',
 'bior2.6',
 'bior2.8',
 'bior3.1',
 'bior3.3',
 'bior3.5',
 'bior3.7',
 'bior3.9',
 'bior4.4',
 'bior5.5',
 'bior6.8',
 'rbio1.1',
 'rbio1.3',
 'rbio1.5',
 'rbio2.2',
 'rbio2.4',
 'rbio2.6',
 'rbio2.8',
 'rbio3.1',
 'rbio3.3',
 'rbio3.5',
 'rbio3.7',
 'rbio3.9',
 'rbio4.4',
 'rbio5.5',
 'rbio6.8',
 'dmey']

# assets forecasting

In [ ]:
# act=['DHR', 'LIN', 'PL', 'GOOG', 'BTCUSD', 'CA', 'NI']

act=['DHR', 'LIN', 'PL', 'GOOG', 'BTCUSD', 'CA', 'NI']

In [ ]:
os.chdir("your directory")
frame=pd.read_csv('close_frame.csv', delimiter=';')

frame_3=frame.copy()
frame_3.index=np.array(frame_3[frame_3.columns[0]])
frame_3=frame_3.iloc[:,1:]

In [ ]:
os.chdir("your directory for predict")


step_smooth=1
len_wind=399
for k in act:

    index_list=frame_3[k].dropna()

    all_fr_tcm=pd.DataFrame()
    all_fr_ctcm=pd.DataFrame()
    err=[]
    err_reason=[]

    for j in modes:
        for i in wavelets:
            while True:
                try:
                    print(k+'_'+i+'_'+j)
                    tmp=perebor_levels_wavedec_without_noise_naiv_1(index_list, len_wind, step_smooth, wavelet=i, modes=j)
    #                 all_fr=pd.concat([all_fr,tmp])
#                     print(all_fr_tcm)
                    clear_output()
                    break
                except Exception as exc:
                    err.append(wavelets)
                    err_reason.append(exc)
                    break
            all_fr_tcm=pd.concat([all_fr_tcm,tmp[0]], axis=1)
            all_fr_ctcm=pd.concat([all_fr_ctcm,tmp[1]], axis=1)
            print(all_fr_tcm)
#             print(i+'_'+j)    
    all_fr_tcm=pd.concat([all_fr_tcm,toch_f(index_list, len_wind, step_smooth)], axis=1)
    all_fr_ctcm=pd.concat([all_fr_ctcm,toch_f(index_list, len_wind, step_smooth)], axis=1)
    print(k)
#     all_fr
    all_fr_tcm.to_csv('ts_TCM_without_noise_2_'+k+'.csv', sep=';', encoding='utf-8')
    all_fr_ctcm.to_csv('ts_CTCM_without_noise_2_'+k+'.csv', sep=';', encoding='utf-8')

# realized volatility forecasting

In [ ]:
act=['MSFT',
 'V',
 'INTC',
 'CMCSA',
 'ADBE',
 'CRM',
 'XEL',
 'AEP',
 'AH',
 'PLD',
 'AMT',
 'AMZN',
 'GOOG',
 'BAC',
 'JPM',
 'BRK+B',
 'BTCUSD',
 'ETHUSD',
 'BZ',
 'CL',
 'CA',
 'PSX',
 'COP',
 'MMM',
 'UNP',
 'XOM',
 'CVX',
 'NFLX',
 'NVDA',
 'GC',
 'LOW',
 'HD',
 'HON',
 'KO',
 'PEP',
 'NI',
 'PA',
 'PL',
 'QUAL',
 'XRPUSD']

In [ ]:
os.chdir("your directory")
frame=pd.read_csv('rv_frame.csv', delimiter=';')
frame_3=frame.copy()
frame_3.index=np.array(frame_3[frame_3.columns[0]])
frame_3=frame_3.iloc[:,1:]

In [ ]:

os.chdir("your directory for predict")

step_smooth=1
len_wind=399
for k in act:

    index_list=np.log(np.sqrt(frame_3[k].dropna()))

    all_fr_tcm=pd.DataFrame()
    all_fr_ctcm=pd.DataFrame()
    err=[]
    err_reason=[]

    for j in modes:
        for i in wavelets:
            while True:
                try:
                    tmp=perebor_levels_wavedec_without_noise_naiv_1(index_list, len_wind, step_smooth, wavelet=i, modes=j)
    #                 all_fr=pd.concat([all_fr,tmp])
    #                 print(i+'_'+j)
                    break
                except Exception as exc:
                    err.append(wavelets)
                    err_reason.append(exc)
                    break
            all_fr_tcm=pd.concat([all_fr_tcm,tmp[0]], axis=1)
            all_fr_ctcm=pd.concat([all_fr_ctcm,tmp[1]], axis=1)
#             print(i+'_'+j)    
    all_fr_tcm=pd.concat([all_fr_tcm,toch_f(index_list, len_wind, step_smooth)], axis=1)
    all_fr_ctcm=pd.concat([all_fr_ctcm,toch_f(index_list, len_wind, step_smooth)], axis=1)
    print(k)
#     all_fr
    all_fr_tcm.to_csv('ts_rv_TCM_without_noise_1_'+k+'.csv', sep=';', encoding='utf-8')
    all_fr_ctcm.to_csv('ts_rv_CTCM_without_noise_1_'+k+'.csv', sep=';', encoding='utf-8')

# without wavelet

In [ ]:
def all_my_meth(index_list, len_wind, step_smooth):
    dates=index_list.index    
    iteratation_5=int((len(index_list)-len_wind)/step_smooth)
#     iteratation_5=5
    name=[]
    frame_prog_my_meth=pd.DataFrame()

    prog1=[]
    prog11=[]
    date1=[]
    
    for i in range(0,iteratation_5):
    
        train=index_list[0+step_smooth*i:len_wind+step_smooth*i]

        tmp=my_method_1_correct(train, len_wind, step_smooth, persentile=0.8)
        
#         TCM
        prog1.append(tmp[0][0])
#         CTCM    
        prog11.append(tmp[1][0])

        date1.append(dates[0+step_smooth*i:len_wind+1+step_smooth*i][-1:][0])      
            
    frame_prog_my_meth['TCM']=prog1
    frame_prog_my_meth['CTCM']=prog11
    frame_prog_my_meth.index=date1
    return frame_prog_my_meth

# assets

In [ ]:
act=['DHR', 'LIN', 'PL', 'GOOG', 'BTCUSD', 'CA', 'NI']

os.chdir("your directory")
frame=pd.read_csv('close_frame.csv', delimiter=';')

frame_3=frame.copy()
frame_3.index=np.array(frame_3[frame_3.columns[0]])
frame_3=frame_3.iloc[:,1:]

In [ ]:
os.chdir("your directory for predict")

step_smooth=1
len_wind=399
for k in act:

    index_list=frame_3[k].dropna()

    all_fr_tcm=pd.DataFrame()

    tmp=all_my_meth(index_list, len_wind, step_smooth)

    
    all_fr_tcm=pd.concat([all_fr_tcm,tmp], axis=1)

    all_fr_tcm=pd.concat([all_fr_tcm,toch_f(index_list, len_wind, step_smooth)], axis=1)
    
    
    print(k)
    all_fr_tcm.to_csv('ts_my_meth_'+k+'.csv', sep=';', encoding='utf-8')


# realized volatility

In [ ]:
act=['MSFT',
 'V',
 'INTC',
 'CMCSA',
 'ADBE',
 'CRM',
 'XEL',
 'AEP',
 'AH',
 'PLD',
 'AMT',
 'AMZN',
 'GOOG',
 'BAC',
 'JPM',
 'BRK+B',
 'BTCUSD',
 'ETHUSD',
 'BZ',
 'CL',
 'CA',
 'PSX',
 'COP',
 'MMM',
 'UNP',
 'XOM',
 'CVX',
 'NFLX',
 'NVDA',
 'GC',
 'LOW',
 'HD',
 'HON',
 'KO',
 'PEP',
 'NI',
 'PA',
 'PL',
 'QUAL',
 'XRPUSD']


os.chdir("your directory")
frame=pd.read_csv('rv_frame.csv', delimiter=';')
frame_3=frame.copy()
frame_3.index=np.array(frame_3[frame_3.columns[0]])
frame_3=frame_3.iloc[:,1:]

In [ ]:

os.chdir("your directory for predict")


step_smooth=1
len_wind=399
for k in act:

    index_list=np.log(np.sqrt(frame_3[k].dropna()))

    all_fr_tcm=pd.DataFrame()

    tmp=all_my_meth(index_list, len_wind, step_smooth)

    
    all_fr_tcm=pd.concat([all_fr_tcm,tmp], axis=1)

    all_fr_tcm=pd.concat([all_fr_tcm,toch_f(index_list, len_wind, step_smooth)], axis=1)
    
    
    print(k)
    all_fr_tcm.to_csv('ts_rv_my_meth_'+k+'.csv', sep=';', encoding='utf-8')